In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import urllib.parse
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [2]:
def generate_url(query, page_number):
    params = {'a': 'hpd', 'q': query, 'start': page_number}
    return "https://wuzzuf.net/search/jobs/?{}".format(urllib.parse.urlencode(params))

In [3]:
def scraping(job):
    skill, requirements, description, title, company, employment, location, level, ExperienceNeeded=[], [], [], [], [], [], [], [], []
    driver = webdriver.Chrome()
    page = 77
    while True:
        driver.get(generate_url(job, page))
    
        time.sleep(2)
        # Find all elements with the specified class
        link_elements = driver.find_elements(By.CLASS_NAME, 'css-m604qf')
        # Initialize an empty list to store the href links
        href_links = []
        # Iterate through each web element and extract the href attribute
        for link_element in link_elements:
            x = link_element.find_element(By.TAG_NAME, 'a')
            job_link = x.get_attribute('href')
            href_links.append(job_link)
        
        if not href_links:
            break
    
        for link in href_links:
            if link != 'None':
                driver.get(link)
                ttitle = driver.find_element(By.CLASS_NAME,"css-f9uh36").text

                empl=driver.find_element(By.CLASS_NAME, "css-ja0r8m").text
                career_level= driver.find_elements(By.CLASS_NAME, "css-4xky9y")

                #location, company
                
                if len(driver.find_elements(By.CLASS_NAME, "css-9geu3q")) == 2:
                    location_company= driver.find_elements(By.CLASS_NAME, "css-9geu3q")[1].text
                else:
                    location_company= driver.find_elements(By.CLASS_NAME, "css-9geu3q")[0].text
                split_parts = location_company.split('-')
                com = split_parts[0].strip()
                loc = split_parts[1].strip()

                #desc, requirements
                
                try:
                    description_elements = driver.find_elements(By.CSS_SELECTOR, ".css-1uobp1k ul li")
                    desc = [li.text for li in description_elements]
                except NoSuchElementException:
                    desc = 'Na' #You need to set a "not found value. It's importa
                
                try:
                    requirements_elements = driver.find_elements(By.CSS_SELECTOR, ".css-1t5f0fr ul li")
                    requirement= [li.text for li in requirements_elements]
                except NoSuchElementException:
                    requirement = 'Na' 
                

                #skills
                e= driver.find_element(By.CLASS_NAME, 'css-s2o0yh')
                elements = e.find_elements(By.CLASS_NAME, 'css-158icaa')
                skills= [element.text for element in elements]



                title.append(ttitle)
                company.append(com)
                location.append(loc)
                level.append(career_level[1].text if len(career_level)>= 1 else "Na")
                ExperienceNeeded.append(career_level[0].text)
                employment.append(empl)
                requirements.append(requirement)
                description.append(desc)
                skill.append(skills)
                
                time.sleep(2)

        page += 1

    dic = {
        "Job Title": title,
        "company":company,
        "Location": location,
        "Employment Type": employment,
        "level":level,
        "requirement":requirements,
        "description":description, 
        "Experience Needed":ExperienceNeeded, 
        "skills":skill
        
    }
    return(dic)
    driver.close()


In [ ]:
software_engineer = scraping('software engineer')

In [ ]:
df_software_engineer= pd.DataFrame(software_engineer)
df_software_engineer.to_csv("software engineer2.csv")